In [ ]:
%run /home/jui/thesis-code/Thesis-code/rai-img-merged/baseline-model.ipynb

In [ ]:
from art.estimators.classification import XGBoostClassifier
from art.attacks.evasion import ZooAttack
from art.estimators.classification import SklearnClassifier

import warnings
warnings.filterwarnings('ignore')

# Select random 7500 samples using positional indexing
subset_indices = np.random.choice(len(y_train_og), 7500, replace=False)
X_train_subset = X_train_og.iloc[subset_indices]  # Use .iloc for positional indexing
y_train_subset = y_train_og[subset_indices]  # Direct positional indexing for NumPy array


def get_adversarial_examples(X_train_subset, pipeline, model):
    """
    Generate adversarial examples using the ZooAttack method.

    Args:
        X_train_subset (np.ndarray): Subset of training data (raw features).
        y_train_subset (np.ndarray): Corresponding labels for the subset.
        pipeline (Pipeline): The original pipeline containing preprocessing and model.

    Returns:
        np.ndarray: Adversarial examples.
    """
    # Extract the preprocessor and model from the pipeline
    preprocessor = pipeline.named_steps['preprocessor']

    # Preprocess the subset of training data using the pipeline's preprocessor
    X_train_subset_preprocessed = preprocessor.transform(X_train_subset)

    # Create ART classifier for XGBoost
    art_classifier = XGBoostClassifier(
        model=model,
        nb_features=X_train_subset_preprocessed.shape[1],
        nb_classes=2)

    # Create the ZooAttack instance
    zoo = ZooAttack(
        classifier=art_classifier,
        confidence=0.0,
        targeted=False,
        learning_rate=1e-1,
        max_iter=50,
        binary_search_steps=10,
        initial_const=1e-3,
        abort_early=True,
        use_resize=False,
        use_importance=False,
        nb_parallel=5,
        batch_size=1,
        variable_h=0.01
    )

    # Generate adversarial examples
    x_train_adv = zoo.generate(X_train_subset_preprocessed)

    return x_train_adv

In [ ]:
x_train_adv = get_adversarial_examples(X_train_subset, pipeline, model)
print(x_train_adv.shape)

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


# Convert all datasets to NumPy arrays before training
xgb_model.fit(x_train_adv, y_train_subset,  verbose=False)

# Predictions on the test set (not the original or adversarial training data)
y_pred_test = xgb_model.predict(X_test_og)

# Compute accuracy on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on Test Set: {accuracy_test * 100:.2f}%")

# Confusion Matrix for test data
cm_test = confusion_matrix(y_test, y_pred_test)

# Plot Confusion Matrix for the test set
plt.figure(figsize=(6, 5))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix - Test Set")
plt.show()
